In [ ]:
# 1. Bibliotheken importieren
import torch
from torchvision import transforms
from model import HockeyActionModel
from dataloader import HockeyDataset
from torch.utils.data import DataLoader

# 2. Device wählen
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Benutze Gerät: {device}")

# 3. Modell vorbereiten
num_classes = 4
model = HockeyActionModel(num_classes=num_classes)
model.load_state_dict(torch.load('models/hockey_action_model.pth', map_location=device))
model = model.to(device)
model.eval()
print("✅ Modell geladen und in Evaluationsmodus gesetzt.")

# 4. Transformationen definieren
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# 5. Testdaten vorbereiten
# Hier kannst du z.B. neue Clips unter "frames_test/" speichern
test_dataset = HockeyDataset('labels.csv', 'frames', transform=transform, frames_per_clip=10)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# 6. Vorhersagen machen
label_map_reverse = {0: 'Check', 1: 'Neutral', 2: 'Schuss', 3: 'Tor'}

with torch.no_grad():
    for frames, labels in test_loader:
        frames = torch.stack(frames).to(device)  # (Batch=1, 10, 3, 224, 224)

        outputs = model(frames)
        _, predicted = torch.max(outputs, 1)

        prediction = label_map_reverse[predicted.item()]
        print(f"✅ Vorhersage: {prediction}")

        break  # Nur 1 Beispiel anzeigen (kannst du rausnehmen für alle)

